In [33]:
import pandas as pd
import json
import networkx as nx
# %pip install scipy==1.8.0

In [34]:
df=pd.read_csv('C:/Users/karti/OneDrive/Desktop/IIA Project/data/tmdb_5000_credits.csv')
df.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [35]:
cast_list = df.cast.tolist()
# print(cast_list[0])
# print(len(cast_list))
crew_list = df.crew.tolist()
# print(cast_list[0])
# print(len(cast_list))
movie_list = df.movie_id.tolist()
# print(movie_list[0])
# print(len(movie_list))

In [36]:
# Creating a mapping of movie_id to cast and crew
m_mapping={}
for i in range(int(len(cast_list)/10)):
  obj1 = json.loads(cast_list[i])
  obj1 = obj1[:3]
  obj2 = json.loads(crew_list[i])
  crew_set = set()
  for crew_i in obj2:
    if crew_i['job'] == 'Producer' or crew_i['job'] == 'Director' or crew_i['job'] == 'Casting':
      crew_set.add((crew_i['id'],crew_i['name'],crew_i['job']))
  for actor_i in obj1:
      crew_set.add((actor_i['id'],actor_i['name'],'actor'))
  m_mapping[movie_list[i]]=list(crew_set)

In [37]:
temp=[]
for item in m_mapping:
  temp = temp + m_mapping[item]
lst = set(temp)
print(lst)

{(2387, 'Patrick Stewart', 'actor'), (23532, 'Jason Bateman', 'actor'), (12922, 'Gary Foster', 'Producer'), (17522, 'Cécile de France', 'actor'), (976, 'Jason Statham', 'actor'), (893, 'Tony Scott', 'Director'), (70851, 'Jack Black', 'actor'), (587020, 'Theo James', 'actor'), (19292, 'Adam Sandler', 'Producer'), (1730, 'Judith Holstra', 'Casting'), (108, 'Peter Jackson', 'Producer'), (36602, 'Peter Berg', 'Director'), (27098, 'Gary Goetzman', 'Producer'), (11874, 'Neal H. Moritz', 'Producer'), (62047, 'Gary Trousdale', 'Director'), (76590, 'Jinko Gotoh', 'Producer'), (64151, 'Mike Mitchell', 'Director'), (42265, 'Peter M. Tobyansen', 'Producer'), (53393, 'Zane Weiner', 'Producer'), (50769, 'Sarah Aubrey', 'Producer'), (1536578, 'Michael J. Travers', 'Producer'), (668, 'Mary Selway', 'Casting'), (4505, 'Michael Bostick', 'Producer'), (54050, 'Mark Waters', 'Director'), (32532, 'Brad Lewis', 'Director'), (21125, 'Richard Kind', 'actor'), (2399, 'Junie Lowry-Johnson', 'Casting'), (227439,

In [38]:
g = nx.Graph()
for item in lst:
    g.add_node(item[0],name=item[1],job=item[2])

In [39]:
def convert_into_dict(temp):
  Dict={}
  for data in temp:
    sorted(temp[data],key=lambda x: x[1])
    for i in range(len(temp[data])-1):
      for j in range(i+1,len(temp[data])):
        if temp[data][i][0]!=temp[data][j][0]:
          if (temp[data][i],temp[data][j]) in Dict.keys():
            Dict[(temp[data][i],temp[data][j])]+=1
          else:
            Dict[(temp[data][i],temp[data][j])]=1
  return Dict

In [40]:
m_mapping_Dict = convert_into_dict(m_mapping)
print(len(m_mapping_Dict))
print(len(lst))
for p in m_mapping_Dict:
  g.add_edge(p[0][0],p[1][0], weight=m_mapping_Dict[p])

14301
1939


In [41]:
# Getting the density of the collaborative graph
print("Density of Graph-> ",nx.density(g))

Density of Graph->  0.00785267964431627


In [42]:
# Create subgraphs for each job
subgraph_actors = g.subgraph([a for a in list(g.nodes) if g.nodes[a]['job']=='actor'])
print("Number of Actor Nodes-> ", subgraph_actors.number_of_nodes())

subgraph_producers = g.subgraph([p for p in list(g.nodes) if g.nodes[p]['job']=='Producer'])
print("Number of Producer Nodes-> ",subgraph_producers.number_of_nodes())

subgraph_directors = g.subgraph([d for d in list(g.nodes) if g.nodes[d]['job']=='Director'])
print("Number of Director Nodes-> ",subgraph_directors.number_of_nodes())

subgraph_casting = g.subgraph([c for c in list(g.nodes) if g.nodes[c]['job']=='Casting'])
print("Number of Casting Director Nodes-> ",subgraph_casting.number_of_nodes())

# Print the total number of nodes in the graph
print("Total Number of Nodes-> ",g.number_of_nodes())

Number of Actor Nodes->  716
Number of Producer Nodes->  642
Number of Director Nodes->  227
Number of Casting Director Nodes->  240
Total Number of Nodes->  1825


In [65]:
if nx.is_connected(g):
    print("The graph is connected")
else:
    print("The graph is not connected")


The graph is not connected


In [ ]:
# Finding the information centrality of each node in the graph for each job

# Th information centranlity does not work for this graph as it is not connected

# g_info=nx.information_centrality(g)
# g_info=list(g_info.items())

# actor_infocen=[(g.nodes[a[0]]['name'],a[1]) for a in g_info if g.nodes[a[0]]['job']=='actor']
# actor_infocen.sort(key=lambda x: x[1],reverse=True)

# producer_infocen=[(g.nodes[p[0]]['name'],p[1]) for p in g_info if g.nodes[p[0]]['job']=='Producer']
# producer_infocen.sort(key=lambda x: x[1],reverse=True)

# director_infocen=[(g.nodes[d[0]]['name'],d[1]) for d in g_info if g.nodes[d[0]]['job']=='Director']
# director_infocen.sort(key=lambda x: x[1],reverse=True)

# casting_infocen=[(g.nodes[c[0]]['name'],c[1]) for c in g_info if g.nodes[c[0]]['job']=='Casting']
# casting_infocen.sort(key=lambda x: x[1],reverse=True)

In [43]:
# Finding the degree of each node in the graph for each job
g_degree = list(g.degree)

actor_degree=[(g.nodes[a[0]]['name'],a[1]) for a in g_degree if g.nodes[a[0]]['job']=='actor']
actor_degree.sort(key=lambda x: x[1],reverse=True)

producer_degree=[(g.nodes[p[0]]['name'],p[1]) for p in g_degree if g.nodes[p[0]]['job']=='Producer']
producer_degree.sort(key=lambda x: x[1],reverse=True)

director_degree = [(g.nodes[d[0]]['name'],d[1]) for d in g_degree if g.nodes[d[0]]['job']=='Director']
director_degree.sort(key=lambda x: x[1],reverse=True)

casting_degree = [(g.nodes[c[0]]['name'],c[1]) for c in g_degree if g.nodes[c[0]]['job']=='Casting']
casting_degree.sort(key=lambda x: x[1],reverse=True)

In [44]:
print('Degree for Actors->')
actor_degree[:5]

Degree for Actors->


[('Angelina Jolie', 86),
 ('Tom Cruise', 79),
 ('Arnold Schwarzenegger', 74),
 ('Matt Damon', 69),
 ('Hugh Jackman', 69)]

In [45]:
print('Degree for Producers->')
producer_degree[:5]

Degree for Producers->


[('Leonardo DiCaprio', 77),
 ('Brad Pitt', 75),
 ('Johnny Depp', 73),
 ('Neal H. Moritz', 70),
 ('Larry J. Franco', 63)]

In [46]:
print('Degree for Directors->')
director_degree[:5]

Degree for Directors->


[('Peter Jackson', 37),
 ('Chris Columbus', 34),
 ('Paul Greengrass', 31),
 ('Robert Zemeckis', 30),
 ('Christopher Nolan', 30)]

In [47]:
print('Degree for Casting Directors->')
casting_degree[:5]

Degree for Casting Directors->


[('Lucinda Syson', 113),
 ('Sarah Finn', 112),
 ('John Papsidera', 103),
 ('Mary Hidalgo', 93),
 ('Ronna Kress', 93)]

In [48]:
# Finding the closeness centrality of each node in the graph for each job
g_closeness = nx.closeness_centrality(g)

g_closeness = list(g_closeness.items())

actor_closeness=[(g.nodes[a[0]]['name'],a[1]) for a in g_closeness if g.nodes[a[0]]['job']=='actor']
actor_closeness.sort(key=lambda x: x[1],reverse=True)

producer_closeness=[(g.nodes[p[0]]['name'],p[1]) for p in g_closeness if g.nodes[p[0]]['job']=='Producer']
producer_closeness.sort(key=lambda x: x[1],reverse=True)

director_closeness=[(g.nodes[d[0]]['name'],d[1]) for d in g_closeness if g.nodes[d[0]]['job']=='Director']
director_closeness.sort(key=lambda x: x[1],reverse=True)

casting_closeness=[(g.nodes[c[0]]['name'],c[1]) for c in g_closeness if g.nodes[c[0]]['job']=='Casting']
casting_closeness.sort(key=lambda x: x[1],reverse=True)

In [49]:
print('Closeness for Actors->')
actor_closeness[:5]

Closeness for Actors->


[('Angelina Jolie', 0.39221533554025845),
 ('Tom Cruise', 0.3796299643339117),
 ('Will Smith', 0.37607584100137126),
 ('Tom Hanks', 0.3690846875468586),
 ('Gary Oldman', 0.3686907871332547)]

In [50]:
print('Closeness for Producers->')
producer_closeness[:5]

Closeness for Producers->


[('Brad Pitt', 0.3842750473235369),
 ('Brian Grazer', 0.3778858756769412),
 ('Neal H. Moritz', 0.3742830634278003),
 ('Johnny Depp', 0.3737970867170089),
 ('Jerry Bruckheimer', 0.3735545713060766)]

In [51]:
print('Closeness for Directors->')
director_closeness[:5]

Closeness for Directors->


[('Michael Mann', 0.35108055644701186),
 ('Christopher Nolan', 0.3478985574459815),
 ('Robert Zemeckis', 0.3447737201036523),
 ('Doug Liman', 0.3427214955792259),
 ('Chris Columbus', 0.3374323769719278)]

In [52]:
print('Closeness for Casting Directors->')
casting_closeness[:5]

Closeness for Casting Directors->


[('Lucinda Syson', 0.40386166418501246),
 ('Sarah Finn', 0.39248269432385785),
 ('Ronna Kress', 0.38651070434533413),
 ('John Papsidera', 0.3841041444783852),
 ('Janet Hirshenson', 0.3834220505481239)]

In [53]:
# Finding the betweenness centrality of each node in the graph for each job
g_betweenness = nx.betweenness_centrality(g)
g_betweenness = list(g_betweenness.items())

actor_betweenness=[(g.nodes[a[0]]['name'],a[1]) for a in g_betweenness if g.nodes[a[0]]['job']=='actor']
actor_betweenness.sort(key=lambda x: x[1],reverse=True)

producer_betweenness=[(g.nodes[p[0]]['name'],p[1]) for p in g_betweenness if g.nodes[p[0]]['job']=='Producer']
producer_betweenness.sort(key=lambda x: x[1],reverse=True)

director_betweenness=[(g.nodes[d[0]]['name'],d[1]) for d in g_betweenness if g.nodes[d[0]]['job']=='Director']
director_betweenness.sort(key=lambda x: x[1],reverse=True)

casting_betweenness=[(g.nodes[c[0]]['name'],c[1]) for c in g_betweenness if g.nodes[c[0]]['job']=='Casting']
casting_betweenness.sort(key=lambda x: x[1],reverse=True)

In [54]:
print('Betweenness for Actors->')
actor_betweenness[:5]

Betweenness for Actors->


[('Tom Hanks', 0.031104455224955606),
 ('Arnold Schwarzenegger', 0.028147915880146295),
 ('Angelina Jolie', 0.026644584336829404),
 ('Tom Cruise', 0.022563294670007703),
 ('Will Smith', 0.018572306306717867)]

In [55]:
print('Betweenness for Producers->')
producer_betweenness[:5]

Betweenness for Producers->


[('Brad Pitt', 0.021984697846443867),
 ('Leonardo DiCaprio', 0.01987926007647464),
 ('Neal H. Moritz', 0.01725756220890224),
 ('Brian Grazer', 0.01711518230386224),
 ('Douglas Wick', 0.016792044719712865)]

In [56]:
print('Betweenness for Directors->')
director_betweenness[:5]

Betweenness for Directors->


[('Brad Bird', 0.006512669632254878),
 ('Paul Verhoeven', 0.005227935012895767),
 ('Chris Buck', 0.004639158308680537),
 ('Chris Columbus', 0.004509760608981904),
 ('Rob Minkoff', 0.0037243096520029307)]

In [57]:
print('Betweenness for Casting Directors->')
casting_betweenness[:5]

Betweenness for Casting Directors->


[('Mary Hidalgo', 0.050085236006448965),
 ('Lucinda Syson', 0.04087103030419086),
 ('Sarah Finn', 0.03589732224501016),
 ('John Papsidera', 0.03557092459283544),
 ('Avy Kaufman', 0.03149010845361329)]

In [58]:
# Finding the clustering coefficient of each node in the graph for each job
g_clustering=nx.clustering(g)
g_clustering=list(g_clustering.items())

actor_clustering = [(g.nodes[a[0]]['name'],a[1]) for a in g_clustering if g.nodes[a[0]]['job']=='actor' and int(a[1])!=1]
actor_clustering.sort(key=lambda x: x[1],reverse=True)

producer_clustering = [(g.nodes[p[0]]['name'],p[1]) for p in g_clustering if g.nodes[p[0]]['job']=='Producer' and int(p[1])!=1]
producer_clustering.sort(key=lambda x: x[1],reverse=True)

director_clustering=[(g.nodes[d[0]]['name'],d[1]) for d in g_clustering if g.nodes[d[0]]['job']=='Director' and int(d[1])!=1]
director_clustering.sort(key=lambda x: x[1],reverse=True)

casting_clustering=[(g.nodes[c[0]]['name'],c[1]) for c in g_clustering if g.nodes[c[0]]['job']=='Casting' and int(c[1])!=1]
casting_clustering.sort(key=lambda x: x[1],reverse=True)


In [59]:
print('Clustering for Actors')
actor_clustering[:5]

Clustering for Actors


[('Tom Hiddleston', 0.9642857142857143),
 ('Josh Duhamel', 0.9454545454545454),
 ('Theo James', 0.9090909090909091),
 ('Shailene Woodley', 0.9090909090909091),
 ('Gwyneth Paltrow', 0.8888888888888888)]

In [60]:
print('Clustering for Producers')
producer_clustering[:5]

Clustering for Producers


[('Bill Miller', 0.9523809523809523),
 ('Nina Jacobson', 0.9523809523809523),
 ('Susan Downey', 0.9454545454545454),
 ('Dan Lin', 0.9454545454545454),
 ('Jerry Weintraub', 0.9333333333333333)]

In [61]:
print('Clustering for Directors')
director_clustering[:5]

Clustering for Directors


[('Guy Ritchie', 0.9454545454545454),
 ('Steven Soderbergh', 0.9333333333333333),
 ('Joe Russo', 0.9166666666666666),
 ('Anthony Russo', 0.9166666666666666),
 ('Joss Whedon', 0.8928571428571429)]

In [62]:
print('Clustering for Casting Directors->')
casting_clustering[:5]

Clustering for Casting Directors->


[('Robin Gurland', 0.9333333333333333),
 ('Marianne Stanicheva', 0.9272727272727272),
 ('Venus Kanani', 0.9090909090909091),
 ('Jennifer Rudnicke', 0.8241758241758241),
 ('Amy Hubbard', 0.7416666666666667)]